In [1]:
import pandas as pd

In [2]:
instance = 'c101'
v = 25
c = 200
n = 101
depot = 0

In [3]:
dados = pd.read_csv(instance+".txt",index_col=0,sep=';').to_dict()

In [4]:
distancia = [[0 for _ in range(n)] for _ in range(n)]

In [5]:
def dist(i,j):
    x = (dados['x'][i]-dados['x'][j])**2
    y = (dados['y'][i]-dados['y'][j])**2
    
    return (x+y)**(1/2)

In [6]:
comeco_horizonte = dados['tempo_preparo'][depot]
fim_horizonte = dados['tempo_entrega'][depot]

In [7]:
M = -1
for i in range(n):
    for j in range(n):
        if i != j:
            distancia[i][j]=round(dist(i,j)) 
            M = max(M,dados['tempo_entrega'][i]+distancia[i][j]-dados['tempo_preparo'][i])
        else: 
            distancia[i][j]=float('inf')

In [23]:
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr

In [24]:
model = Model("vrptw")

### Variáveis

In [25]:
travels = model.binary_var_cube(v,n,n,'travel')#matriz tridimensional k*v²

In [26]:
service = {(k,i): model.continuous_var(
    lb=dados['tempo_preparo'][i],
        ub=dados['tempo_entrega'][i],name=f'service_start_{k}_{i}') 
           for k in range(v) for i in range(n)}#momento em que o serviço começa no cliente i

### Função Objetiva

In [27]:
model.minimize(model.sum(travels[k,i,j]*distancia[i][j] for i in range(n) for j in range(n) for k in range(v)))

### Restrições

In [28]:
#Visitar o cliente i é obrigatório
for i in range(1,n):
    model.add_constraint(
     model.sum(travels[k,i,j]
       for k in range(v)
           for j in range(n))==1,
            'Visit_Client_'+str(i))

In [29]:
#Respeitar Capacidade
for k in range(v):
    model.add_constraint(
        model.sum(travels[k,i,j]*dados['demanda'][i]
            for i in range(n)
                for j in range(n)) <= c,
                    'Capacity_Vehicle_'+str(k))

In [30]:
#Saída a Partir do Depósito
for k in range(v):
    model.add_constraint(
        model.sum(travels[k,0,j]
            for j in range(n)) == 1,
                'Exit_Depot_Vehicle_'+str(k))

In [31]:
#Conservação de Fluxo
for k in range(v):
    for i in range(n):
        model.add_constraint(
            model.sum(travels[k,i,j]-travels[k,j,i]
                    for j in range(i+1,n)) == 0,
                        f'Flux_Conservation_Vehicle_{k}_Node_{i}')

In [32]:
#Tempo de Saida
for k in range(v):
    for i in range(n):
        for j in range(i+1,n):
            model.add_constraint(service[k,i]+distancia[i][j]-M*(1-travels[k,i,j]) <= service[k,j],
                                f'Exit_Time_{k}_{i}_{j}')

In [33]:
#Forbidden Arcs
model.add_constraint(model.sum(travels[k,i,i] for k in range(v) for i in range(n))==0);

In [ ]:
import time
start = time.time() #Tempo de ínicio
subcicle = True     #Condição de parada
#Assumindo que existe subciclos, nós...
while subcicle == True:

    #Rota percorrida por cada veículo
    routes = {x:{} for x in range(v)}    

    #Solução VRP - Dicionário Desordenado
    _Solution = model.solve(log_output=True).as_dict()
    #saida veículo
    #    |   |
    #    v   v
    #Arc_i_j_k: True/False <-Valores Possíveis
    #      ^
    #      |
    #   chegada
    #
    #Arc_i_j_k = Chave

    for i in _Solution:
        #
        #
        #"Arc_i_j_k"
        #     |   
        #     v                      0   1   2
        aux = i.split("_")[1:] #<- ["i","j","k"]
        #        ^
        #        |
        #["Arc","i","j","k"]
        #
        #         veiculo=k    saida=i      chegada=j
        #             |            |            |
        #             v            v            v
        routes[int(aux[2])][int(aux[0])]=int(aux[1])
        # ^
        # |_ Rotas agrupadas por veículo

    print(routes)
    count = 0 #Contador para as eliminações de subciclo
    summ = 0 #Condição de parada

    #O trecho de código abaixo deve remover o caminho
    #principal para futuramente verificarmos se existe
    #algum subciclo na rota do veículo 'k'
    for k in veiculos.index:

        #Primeiramente, verificamos se o veículo 'k'
        #percorreu, ou não, pelo menos um arco
        if len(routes[k]) > 0:

            #Se ele percorreu pelo menos um arco:
            #Começamos verificando qual o ponto de saida
            #do veículo 'k' e atribuímos esse nodo para
            #o ponto de partida do nosso loop

            inicial = veiculos.loc[k]["Inicial"]
            #  ^
            #  |
            #Saída de 'k'
            #
            #Chegada de 'k'
            #saindo do ponto
            #inicial
            #  |
            #  v
            actual_node = routes[k].pop(inicial)

            #inicial->actual_node = primeiro arco de todas as rotas

            #Iremos percorrer a rota até que alcançemos a vítima
            while actual_node not in vitimas["Ponto"].values:
                # actual_node                actual_node
                #vira chegada do        inicia como saída do
                #  arco                        arco
                #   |                            |
                #   v                            v
                actual_node = routes[k].pop(actual_node)

                #it0: actual_node-> nodo 'j'
                #               ^
                #               |
                #it1:     actual_node -> nodo 'k'
                #                             ^
                #                             |
                #it2:                     actual_node -> nodo 'l' ...


        #Se sobrou algum subciclo
        #depois da remoção do caminho
        #principal então 
        #len(routes[k]) > 0
        summ += len(routes[k])


        #Se sobrou algum arco dentro
        #da rota, então deve ser algum
        #subciclo que devemos retirar
        if len(routes[k]) >0:

            #Enquanto ainda existir algum arco
            #dentro da rota do veículo 'k'
            #ele faz parte de um subciclo
            #que deve ser retirado, portanto
            #iteramos até removermos todos os
            #subciclos
            while(len(routes[k])>0):
                print(routes[k])
                cut = LinearExpr(vrp) #Expressão Linear do Subciclo atual

                first_node = list(routes[k].keys())[0]
                #  ^
                #  |
                #saída do primeiro arco
                #do subciclo atual
                #
                #chegada do primeiro
                #arco do subciclo atual
                #  |
                #  v
                actual_node = routes[k].pop(first_node)

                #Adicionamos o arco inicial à restrição
                cut += tavels[k,int(first_node),int(actual_node)]

                #Tamanho do subciclo -1
                size = 0
                while(actual_node != first_node):
                    next_node = routes[k].pop(actual_node)                 #Percorre o subciclo e o adiciona ao#
                    cut += tavels[k,int(actual_node),int(next_node)]#conjunto de subciclos proíbidos.#
                    actual_node = next_node
                    size+=1
                count+=1
                vrp.add_constraint(cut<=size,"SubCycleCut_"+str(count))

    if summ == 0:
        subcicle = False
print(time.time()-start)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 120275 rows and 122239 columns.
MIP Presolve modified 178475 coefficients.
Aggregator did 700 substitutions.
Reduced MIP has 7951 rows, 134611 columns, and 352268 nonzeros.
Reduced MIP has 133801 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.75 sec. (1017.24 ticks)
Tried aggregator 2 times.
Detecting symmetries...
MIP Presolve eliminated 18 rows and 323 columns.
MIP Presolve added 120 rows and 0 columns.
MIP Presolve modified 3611 coefficients.
Aggregator did 31 substitutions.
Reduced MIP has 8022 rows, 134257 columns, and 351508 nonzeros.
Reduced MIP has 133453 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.48 sec. (573.21 ticks)
Probing time = 0.29 sec. (55.29 ticks)
Clique table members: 219295.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mod


Performing restart 2

Repeating presolve.
Tried aggregator 1 time.
Reduced MIP has 8022 rows, 134239 columns, and 355172 nonzeros.
Reduced MIP has 133435 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.16 sec. (198.07 ticks)
Tried aggregator 1 time.
Reduced MIP has 8022 rows, 134239 columns, and 355172 nonzeros.
Reduced MIP has 133435 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.21 sec. (258.51 ticks)
Represolve time = 3.78 sec. (1062.22 ticks)


In [ ]:
print(model.solve())

In [ ]:
i